In [67]:
import sys
import os
sys.path.append('../')
os.chdir('../')

import pandas as pd
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime
import time
import threading
import json
import requests
import threading
import re
from src import utils  

# from src.umls import umls_api as umlsAPI


In [68]:
umls_api_key = "577c8f76-500d-4e94-9139-8a798bdbe78f"
umls_url = "https://uts-ws.nlm.nih.gov/rest"
timeout_value = 10

#### Peticion dado un texto, nos devuelve las concidencias.

In [69]:
class MyUmlsAPI:
    # TODO:

    """API para UMLS.

            Atributos
            ----------

            Constructor
            ------------
                MyUmlsAPI()

            Métodos
            -------
                create_url: 
                request_text_umls:
    """

    def __init__(self):
        pass

    def create_url(self,kind,list_var={}):

        """Dado el tipo de url, crea el link para realizar la peticion.

            Parametros
            ----------
                kind: String
                    Tipo de petición a realizar en la API. Ex: Eutirox

            Retorna
            -------
                url: String
                    Url para realizar la peticion, Ex: https://uts-ws.nlm.nih.gov/rest/search/current?string=dermatitis alérgica&sabs=MDRSPA,SCTSPA,MSHSPA&returnIdType=code&apiKey=<key>&pageNumber=1&searchType=exact

        """

        url = umls_url + kind

        for index, (key, value) in enumerate(list_var.items()):
            if isinstance(value,int):
                url += "&" + key + "=" + str(value)
            elif isinstance(value,str):
                url += "&" + key + "=" + value

        return url

    def request_text_umls(self,text):
        """Devuelve las coincidencias con el texto proporcionado para encontrar reacciones adversas.

            Parametros
            ----------
                text: String
                    Texto a buscar en el tesaurio. Ex: Eutirox

            Retorna
            -------
                text: json
                    Contiene las coincidencias encontradas que coincide con el texto proporcionado.

                resp_code: int
                    Codigo de respuesta de nuestra consulta. Ex: 200
        """

        join_path = "/search/current?"

        dic_var = {
            "string": text,
            "sabs": "MDRSPA,SCTSPA,MSHSPA",
            "returnIdType": "code",
            'pageNumber': 1,
            "searchType": "exact",
            "apiKey":umls_api_key
        }

        url = self.create_url(join_path,dic_var)

        response = requests.get(url,timeout=timeout_value)
        resp_code = response.status_code
        resp_message = response.json()

        return resp_message,resp_code

In [66]:
umls = MyUmlsAPI()

In [70]:
text = "trastorno gastrointestinal"
resp_message, resp_code = umls.request_text_umls(text)

In [71]:
resp_code

200

In [72]:
resp_message

{'pageSize': 25,
 'pageNumber': 1,
 'result': {'classType': 'searchResults',
  'results': [{'ui': '10017944',
    'rootSource': 'MDRSPA',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/source/MDRSPA/10017944',
    'name': 'Trastorno gastrointestinal'},
   {'ui': '10013225',
    'rootSource': 'MDRSPA',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/source/MDRSPA/10013225',
    'name': 'Trastorno gastrointestinal'}],
  'recCount': 2}}

Ejemplo de respuesta

#### Tesaurios de los que sacamos informacion.

- CPTSP	                CPT Spanish	                                    2001AC	SPA	3
- ICPCSPA	            ICPC Spanish	                                2000AA	SPA	0
- LNC-ES-ES	            LOINC Linguistic Variant - Spanish, Spain	    2023AB	SPA	0
- MDRSPA	            MedDRA Spanish	                                2023AB	SPA	3
- MEDLINEPLUS_SPA	    MedlinePlus Spanish Health Topics	            2023AA	SPA	0
- MSHSPA	            MeSH Spanish	                                2023AA	SPA	3
- SCTSPA	            SNOMED CT Spanish Edition	                    2023AB	SPA	9
- WHOSPA	            WHOART Spanish	                                1999AA	SPA	2

[Lincencias](https://uts.nlm.nih.gov/uts/license/license-category-help.html?_gl=1*1p2hsly*_ga*OTc0MzYyODE0LjE3MDE2NTg3NjY.*_ga_7147EPK006*MTcwMjI1MDkwNC4xMC4xLjE3MDIyNTU5MTcuMC4wLjA.*_ga_P1FPTH9PL4*MTcwMjI1MDkwNC4xMC4xLjE3MDIyNTU5MTcuMC4wLjA.)

- LNC-ES-AR	            LOINC Linguistic Variant - Spanish, Argentina	2023AB	SPA	0
- LNC-ES-MX	            LOINC Linguistic Variant - Spanish, Mexico	    2023AB	SPA	0


MDRSPA,SCTSPA,MSHSPA,CPTSP,ICPCSPA,LNC-ES-ES,MEDLINEPLUS_SPA,WHOSPA

MDRSPA,SCTSPA, son las que cuentan con mas informacion, podemos añadir MSHSPA

#### Texto limpio para su analisis.

In [58]:
texto_analizar = [
    ['disulfiram', 'ser', 'mismo', 'sustancia', 'relativamente', 'tóxico', 'observar', 'escaso', 'efecto', 'secundario', 'administrar', 'sólo', 'dosis', 'terapéutico', 'animal', 'ser', 'humano'],
    ['efecto', 'secundario', 'reacción', 'disulfiramalcohol', 'haber', 'describir', 'sección'],
    ['mayoría', 'reacción', 'disulfiram', 'ser', 'autolimitada', 'representar', 'riesgo', 'vital', 'paciente'],
    ['número', 'reducido', 'paciente', 'poder', 'experimentar', 'adormecer', 'leve', 'transitorio', 'fatigo', 'impotencia', 'cefalea', 'erupción', 'acneiform', 'dermatitis', 'alérgico', 'sabor', 'metálico', 'ajo', 'dos', 'primero', 'semana', 'tratamiento', 'síntoma', 'soler', 'desaparecer', 'espontáneamente', 'continuación', 'tratamiento', 'reducción', 'posología'],
    ['reacción', 'adverso', 'ordenado', 'según', 'frecuencia', 'aparición'],
    ['trastorno', 'psiquiátrico'],
    ['rara', 'episodio', 'psicótico', 'incluir', 'paranoia', 'esquizofrenia'],
    ['frecuente', 'depresión', 'manir'],
    ['trastorno', 'sistema', 'nervioso'],
    ['frecuente', 'somnolencia', 'cefalear', 'neuritis', 'periférico', 'neuritis', 'óptico', 'polineuritis'],
    ['frecuente', 'fatiga', 'reacción', 'distónica'],
    ['rara', 'cambio', 'humor', 'crisis', 'convulsiva', 'confusión'],
    ['trastorno', 'ocular'],
    ['rara', 'cambio', 'visión'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'edema', 'papilarceguero'],
    ['trastorno', 'gastrointestinal'],
    ['frecuente', 'nausea', 'vómito', 'diarreo'],
    ['trastorno', 'hepatobiliar'],
    ['frecuente', 'alteración', 'determinación', 'función', 'hepático', 'aumento', 'transaminasa', 'bilirrubina'],
    ['rara', 'hepatitis', 'hepatotoxicidad', 'daño', 'hepatocelular', 'hepatitis', 'fulminante', 'necrosis', 'hepático', 'poder', 'originar', 'fallo', 'hepático', 'coma', 'hepático', 'muertir', 'efecto', 'sistema', 'hepatobiliar', 'generalmente', 'aparecer', 'primero', 'mes', 'tratamiento'],
    ['trastorno', 'piel', 'tejido', 'subcutáneo'],
    ['frecuente', 'dermatitis', 'alérgico'],
    ['rara', 'coloración', 'amarillentar', 'piel'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'erupción', 'acneiform', 'haber', 'describir', 'caso', 'fototoxicidad'],
    ['trastorno', 'aparato', 'reproductor', 'mama'],
    ['rara', 'reducción', 'líbido'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'disfunción', 'eréctil'],
    ['trastorno', 'congénito'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'haber', 'describir', 'alguno', 'caso', 'malformación', 'congénita', 'tratamiento', 'disulfiram'],
    ['trastorno', 'metabolismo'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'elevación', 'colesterol'],
    ['trastorno', 'cardíaco'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'taquicardia', 'cambio', 'ecg', 'asociado'],
    ['trastorno', 'vascular'],
    ['frecuencia', 'conocido', 'poder', 'estimar', 'partir', 'dato', 'disponible', 'hipertensión', 'arterial'],
    ['trastorno', 'general'],
    ['frecuente', 'adormecimiento', 'fatigar', 'malestar', 'general', 'halitosis', 'dolor', 'abdominal', 'pérdida', 'gusto', 'sabor', 'metálico', 'ajo'],
    ['paciente', 'sometido', 'mantenimiento', 'disulfiram', 'mgdía', 'tender', 'desarrollar', 'neuropatía', 'periférico', 'neuropatía', 'mejora', 'retirar', 'tratamiento', 'disulfiram'],
    ['regla', 'general', 'erupción', 'cutánea', 'ocasional', 'controlar', 'fácilmente', 'mediante', 'administración', 'simultáneo', 'antihistamínico'],
    ['notificación', 'sospecha', 'reacción', 'adverso'],
    ['ser', 'importante', 'notificar', 'sospecha', 'reacción', 'adverso', 'medicamento', 'tras', 'autorización', 'permitir', 'supervisión', 'continuado', 'relación', 'beneficioriesgo', 'medicamento', 'invitar', 'profesional', 'sanitario', 'notificar', 'sospecha', 'reacción', 'adverso', 'través', 'sistema', 'español', 'farmacovigilancia', 'medicamento', 'uso', 'humano', 'websiwwwnotificarames'],
]

In [59]:
pairs = utils.get_pairs(texto_analizar)

In [ ]:
reacciones_adversas = []
for pair in pairs:
    text = pair[0] + " " + pair[1]
    response, code =  umls.request_text_umls(text)
    # print(response["result"]["results"])
    if response["result"]["results"] is not None:
        for index,element in enumerate(response["result"]["results"]):
            # print(element["name"])
            reacciones_adversas.append(element["name"])
    else:
        continue
        # print("Not element found")

In [64]:
reacciones_adversas

['Humanos',
 'sabor metálico',
 'Trastornos del Gusto',
 'Sabor metálico',
 'Episodio psicótico',
 'estructura del sistema nervioso',
 'Sistema Nervioso',
 'Reacción distónica',
 'Crisis convulsiva',
 'Trastorno gastrointestinal',
 'Trastorno gastrointestinal',
 'daño hepatocelular',
 'hepatitis fulminante',
 'Necrosis Hepática Masiva',
 'Hepatitis fulminante',
 'Fallo Hepático',
 'Fallo hepático',
 'coma hepático',
 'Coma hepático',
 'Coma hepático',
 'estructura del tejido celular subcutáneo',
 'Tejido Subcutáneo',
 'estructura anatómica del aparato reproductor',
 'disfunción eréctil',
 'Disfunción Eréctil',
 'Disfunción eréctil',
 'malformación congénita',
 'Trastorno vascular',
 'Trastorno vascular',
 'hipertensión arterial',
 'Hipertensión arterial',
 'Hipertensión arterial',
 'malestar general',
 'Malestar general',
 'dolor abdominal',
 'Dolor Abdominal',
 'Dolor abdominal',
 'Dolor abdominal',
 'sabor metálico',
 'Trastornos del Gusto',
 'Sabor metálico',
 'erupción cutánea',
 '

#### Eliminamos duplicados al realizar la request a diferences tesaurios.

In [73]:
lista_sin_duplicados = utils.remove_duplicated(reacciones_adversas)

lista_sin_duplicados

['malformación congénita',
 'trastorno vascular',
 'fallo hepático',
 'exantema',
 'malestar general',
 'sistema nervioso',
 'trastornos del gusto',
 'hepatitis fulminante',
 'hipertensión arterial',
 'humanos',
 'necrosis hepática masiva',
 'estructura del tejido celular subcutáneo',
 'daño hepatocelular',
 'trastorno gastrointestinal',
 'crisis convulsiva',
 'tejido subcutáneo',
 'erupción cutánea',
 'episodio psicótico',
 'estructura anatómica del aparato reproductor',
 'reacción distónica',
 'disfunción eréctil',
 'dolor abdominal',
 'estructura del sistema nervioso',
 'sabor metálico',
 'coma hepático']